<a href="https://colab.research.google.com/github/Rustydinoco/ML_Learning/blob/master/Customer_Churn_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline

In [18]:
class Model(nn.Module):
  def __init__(self,in_features = 13,h1= 16,h2 = 19,h3 = 22,out_features = 2):
    super().__init__()
    self.fc1 = nn.Linear(in_features,h1)
    self.fc2 = nn.Linear(h1,h2)
    self.fc3 = nn.Linear(h2,h3)
    self.fc4 = nn.Linear(h3,out_features)

  def forward(self,x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = self.fc4(x)
    return x

In [19]:
torch.manual_seed(41)

model = Model()

In [20]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Dataset ML/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [22]:
df = df.drop(['customerID','gender','SeniorCitizen','Partner','Dependents','tenure','PaperlessBilling'], axis = True)
df.head()

,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Electronic check,29.85,29.85,No
1,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,Mailed check,56.95,1889.5,No
2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Mailed check,53.85,108.15,Yes
3,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,Bank transfer (automatic),42.30,1840.75,No
4,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Electronic check,70.70,151.65,Yes


In [23]:
df['PhoneService'] = df['PhoneService'].replace('No', 0.0)
df['PhoneService'] = df['PhoneService'].replace('Yes', 1.0)

df['MultipleLines'] = df['MultipleLines'].replace('No', 0.0)
df['MultipleLines'] = df['MultipleLines'].replace('Yes', 1.0)
df['MultipleLines'] = df['MultipleLines'].replace('No phone service', 0.2)

df['OnlineSecurity'] = df['OnlineSecurity'].replace('No', 0.0)
df['OnlineSecurity'] = df['OnlineSecurity'].replace('Yes', 1.0)
df['OnlineSecurity'] = df['OnlineSecurity'].replace('No internet service', 0.2)

df['InternetService'] = df['InternetService'].replace('DSL', 0.0)
df['InternetService'] = df['InternetService'].replace('Fiber optic', 1.0)
df['InternetService'] = df['InternetService'].replace('No', 0.2)

df['OnlineBackup'] = df['OnlineBackup'].replace('No', 0.0)
df['OnlineBackup'] = df['OnlineBackup'].replace('Yes', 1.0)
df['OnlineBackup'] = df['OnlineBackup'].replace('No internet service', 0.2)

df['DeviceProtection'] = df['DeviceProtection'].replace('No', 0.0)
df['DeviceProtection'] = df['DeviceProtection'].replace('Yes', 1.0)
df['DeviceProtection'] = df['DeviceProtection'].replace('No internet service', 0.2)

df['TechSupport'] = df['TechSupport'].replace('No', 0.0)
df['TechSupport'] = df['TechSupport'].replace('Yes', 1.0)

df['StreamingTV'] = df['StreamingTV'].replace('No', 0.0)
df['StreamingTV'] = df['StreamingTV'].replace('Yes', 1.0)
df['StreamingTV'] = df['StreamingTV'].replace('No internet service', 0.2)

df['StreamingMovies'] = df['StreamingMovies'].replace('No', 0.0)
df['StreamingMovies'] = df['StreamingMovies'].replace('Yes', 1.0)

df['Contract'] = df['Contract'].replace('Month-to-month', 0.0)
df['Contract'] = df['Contract'].replace('One year', 1.0)
df['Contract'] = df['Contract'].replace('Two year', 2.0)

df['PaymentMethod'] = df['PaymentMethod'].replace('Electronic check', 0.0)
df['PaymentMethod'] = df['PaymentMethod'].replace('Mailed check', 1.0)
df['PaymentMethod'] = df['PaymentMethod'].replace('Bank transfer (automatic)', 2.0)
df['PaymentMethod'] = df['PaymentMethod'].replace('Credit card (automatic)', 3.0)

df['Churn'] = df['Churn'].replace('No', 0.0)
df['Churn'] = df['Churn'].replace('Yes', 1.0)



<ipython-input-23-002971ee7243>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['PhoneService'] = df['PhoneService'].replace('Yes', 1.0)
<ipython-input-23-002971ee7243>:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['MultipleLines'] = df['MultipleLines'].replace('No phone service', 0.2)
<ipython-input-23-002971ee7243>:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the fu

In [24]:
df.head()

,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0.0,0.2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,29.85,29.85,0.0
1,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,56.95,1889.5,0.0
2,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,53.85,108.15,1.0
3,0.0,0.2,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,2.0,42.30,1840.75,0.0
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.70,151.65,1.0


In [25]:
X = df.drop('Churn', axis = 1)
y = df['Churn']

In [26]:
X = X.values
y = y.values


In [27]:
import sklearn.model_selection
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=41)

In [28]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.